In [1]:
import gymnasium as gym
from tinympc.tinympc_torch import MPCSolver, LinearDynamics, LinearCost, LinearConstraints
import numpy as np
import torch

In [2]:
env = gym.make('Pendulum-v1', render_mode="human")
observation, info = env.reset()
env.state[0] = -0.2

g = 10
m = 1
l = 1

dt = 0.05
N = 10
num_envs = 1

A = torch.tensor([[1, dt], [3 * g / (2 * l), 1]]).unsqueeze(0).repeat(num_envs,1,1)
B = torch.tensor([[0],[3./(m*l**2)*dt]]).unsqueeze(0).repeat(num_envs,1,1)
dyn = LinearDynamics(A,B)

Q = torch.eye(2).unsqueeze(0).repeat(num_envs,1,1)
R = torch.eye(1).unsqueeze(0).repeat(num_envs,1,1)
Qf = torch.eye(2).unsqueeze(0).repeat(num_envs,1,1)
cost = LinearCost(Q,R,Qf)


xlb = torch.ones((2,1)).unsqueeze(0).repeat(num_envs,1,1) * -0.2
xub = torch.ones((2,1)).unsqueeze(0).repeat(num_envs,1,1) * 0.2
ulb = torch.ones((1,1)).unsqueeze(0).repeat(num_envs,1,1) * -2
uub = torch.ones((1,1)).unsqueeze(0).repeat(num_envs,1,1) * 2
constraints = LinearConstraints(xlb,xub,ulb,uub)

mpc = MPCSolver(dyn, cost, constraints, N, 0.001, num_envs, 'cpu')


for i in range(1000):
    action = env.action_space.sample()  # agent policy that uses the observation and info

    x0 = env.state.copy()
    xref = np.zeros((2,N))

    x0 = torch.tensor(x0).reshape(2,1)
    x0 = x0.unsqueeze(0).repeat(num_envs,1,1)
    xref =  torch.tensor(xref).unsqueeze(0).repeat(num_envs,1,1)

    mpc.solve(xref,x0)
    action = mpc.u[0,:,0].numpy()

    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break

env.close()

/home/infas12/anaconda3/envs/robo831/lib/python3.8/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.state to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.state` for environment variables or `env.get_wrapper_attr('state')` that will search the reminding wrappers.
  logger.warn(


Kinf: 
 tensor([[[153.3789,   8.8608]]])
Pinf: 
 tensor([[[33236.0312,  1917.6387],
         [ 1917.6392,   111.6657]]])
